Neighborhoods in Toronto - Segmenting and Clustering Assignment

Author: Panagiotis Sidiropoulos

In [1]:
#!conda update --all
#print('libs have been updated')

import numpy as np
import pandas as pd
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
import json

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes
import folium
from bs4 import BeautifulSoup
import requests
print('import step done!')

C:\Users\sidpa\Anaconda3\lib\site-packages\scipy\__init__.py:115: UserWarning: Numpy 1.13.3 or above is required for this version of scipy (detected version 1.11.3)
  UserWarning)


import step done!


Having imported all needed libraries, I utilize the BeautifulSoup library to extract the data from the given url for Toronto

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r = requests.get(url).text
soup = BeautifulSoup(r,'html5lib')
tbl = soup.find_all('table')
tbl_rows = soup.find_all('tr')
tbl_el = soup.find_all('td')

I create the dataframe, with 3 columns as asked. Then I drop the first row as it contained the headers.
I form another table which drops rows that the Borough is not assigned. Likewise, I assign each Neighborhood value to Borough's value if it's not assigned.
Finally I reset Index as I have dropped many rows.

In [3]:
import pandas as pd

data_table = pd.read_html(str(tbl), flavor = 'bs4')[0]
data_table.columns = ['Postal Code', 'Borough', 'Neighborhood']
data_table = data_table[1:]
tbl_tor = data_table[~(data_table['Borough'] == 'Not assigned')]

for i,place in enumerate(tbl_tor['Neighborhood']):
    if (place == 'Not assigned'):
        tbl_tor['Neighborhood'] = tbl_tor['Borough']

tbl_tor.reset_index(drop=True, inplace=True)

In [4]:
tbl_tor.shape

(103, 3)

Step 2: Import geopandas and geopy _
        Append Latitude and Longitude to table.

In [6]:
!conda install -c conda-forge geopandas --yes
!conda install -c conda-forge geopy --yes

Solving environment: ...working... 
  - https://repo.continuum.io/pkgs/main/win-64/conda-build-3.0.27-py36h309a530_0.tar.bz2/win-64::conda-build-3.0.27-py36h309a530_0, https://repo.continuum.io/pkgs/main/win-64/console_shortcut-0.1.1-h6bb2dd7_3.tar.bz2/win-64::console_shortcut-0.1.1-h6bb2dd7_3, https://repo.continuum.io/pkgs/main/win-64/wincertstore-0.2-py36h7fe50ca_0.tar.bz2/win-64::wincertstore-0.2-py36h7fe50ca_0
  - defaults/win-64::conda-build-3.0.27-py36h309a530_0, https://repo.continuum.io/pkgs/main/win-64/console_shortcut-0.1.1-h6bb2dd7_3.tar.bz2/win-64::console_shortcut-0.1.1-h6bb2dd7_3, https://repo.continuum.io/pkgs/main/win-64/wincertstore-0.2-py36h7fe50ca_0.tar.bz2/win-64::wincertstore-0.2-py36h7fe50ca_0
  - defaults/win-64::wincertstore-0.2-py36h7fe50ca_0, https://repo.continuum.io/pkgs/main/win-64/conda-build-3.0.27-py36h309a530_0.tar.bz2/win-64::conda-build-3.0.27-py36h309a530_0, https://repo.continuum.io/pkgs/main/win-64/console_shortcut-0.1.1-h6bb2dd7_3.tar.bz2/win-64:

Step 2: Get Coordinates from google API

In [7]:
import geopy
import geopandas

locator = Nominatim(user_agent="myGeocoder")

for i,place in enumerate(tbl_tor['Borough']):
    location = locator.geocode(place)
    tbl_tor.loc[i,'Latitude'] = location.latitude
    tbl_tor.loc[i,'Longitude'] = location.longitude